In [1]:
from phase1_model_instantiation import model_setup, func_test

In [2]:
print("Available Qwen models:")
models = model_setup.list_qwen_models()
for name, info in models.items():
    print(f"  {name}: modified {info['last_modified']}")

Available Qwen models:
  qwen-2.5-3b: modified 2025-09-04 22:55:50+00:00


In [4]:
import os
model_path = "/workspace/praxis-research/base-model/qwen-2.5-3b"
print("Files in model directory:")
for file in os.listdir(model_path):
    print(f"  {file}")

# Check if config.json exists and what's in it
config_path = os.path.join(model_path, "config.json")
if os.path.exists(config_path):
    with open(config_path, 'r') as f:
        import json
        config = json.load(f)
        print(f"\nConfig model_type: {config.get('model_type', 'MISSING')}")
else:
    print("\n❌ config.json is missing!")

Files in model directory:
  cache

❌ config.json is missing!


In [5]:
# List all blobs in the qwen_models directory

secrets = model_setup.get_secrets()
from azure.storage.blob import BlobServiceClient

connection_string = f"DefaultEndpointsProtocol=https;AccountName={secrets['storage_account_name']};AccountKey={secrets['storage_account_key']};EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(secrets['container'])

print("All blobs in qwen_models/:")
blobs = list(container_client.list_blobs(name_starts_with="qwen_models/"))
for blob in blobs:
    print(f"  {blob.name} ({blob.size} bytes)")

All blobs in qwen_models/:
  qwen_models/qwen-2.5-3b/cache/.locks/models--Qwen--Qwen2.5-3B-Instruct/07bfe0640cb5a0037f9322287fbfc682806cf672.lock (0 bytes)
  qwen_models/qwen-2.5-3b/cache/.locks/models--Qwen--Qwen2.5-3B-Instruct/20024bfe7c83998e9aeaf98a0cd6a2ce6306c2f0.lock (0 bytes)
  qwen_models/qwen-2.5-3b/cache/.locks/models--Qwen--Qwen2.5-3B-Instruct/38047c6284a25427043f0ab040f623a2a20dd093.lock (0 bytes)
  qwen_models/qwen-2.5-3b/cache/.locks/models--Qwen--Qwen2.5-3B-Instruct/443909a61d429dff23010e5bddd28ff530edda00.lock (0 bytes)
  qwen_models/qwen-2.5-3b/cache/.locks/models--Qwen--Qwen2.5-3B-Instruct/4783fe10ac3adce15ac8f358ef5462739852c569.lock (0 bytes)
  qwen_models/qwen-2.5-3b/cache/.locks/models--Qwen--Qwen2.5-3B-Instruct/51b83fbd4e2d363b60235b9c8d494046a83c1cbf.lock (0 bytes)
  qwen_models/qwen-2.5-3b/cache/.locks/models--Qwen--Qwen2.5-3B-Instruct/67347b23fb4165b652eb6611f5e1f2a06dfcddba8e909df1b2b0b1857bee06c2.lock (0 bytes)
  qwen_models/qwen-2.5-3b/cache/.locks/models-

In [3]:
# Load model/tokenizer from the correct cache path
model_path = "/workspace/praxis-research/base-model/qwen-2.5-3b/cache/models--Qwen--Qwen2.5-3B-Instruct/snapshots/aa8e72537993ba99e69dfaafa59ed015b17504d1"

model, tokenizer = model_setup.get_qwen_model(model_path)

✅ Using cached Qwen model at /workspace/praxis-research/base-model/qwen-2.5-3b/cache/models--Qwen--Qwen2.5-3B-Instruct/snapshots/aa8e72537993ba99e69dfaafa59ed015b17504d1
🔄 Loading model and tokenizer from /workspace/praxis-research/base-model/qwen-2.5-3b/cache/models--Qwen--Qwen2.5-3B-Instruct/snapshots/aa8e72537993ba99e69dfaafa59ed015b17504d1


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Successfully loaded Qwen model /workspace/praxis-research/base-model/qwen-2.5-3b/cache/models--Qwen--Qwen2.5-3B-Instruct/snapshots/aa8e72537993ba99e69dfaafa59ed015b17504d1


In [4]:
# Start interactive chat
func_test.chat_loop(model, tokenizer, system_text="You are a helpful AI assistant.", max_new_tokens=256)


Interactive QA/chat with Qwen. Type 'exit' or 'quit' to stop.



You:  are you there?


Qwen: Yes, I am here and ready to help you. How can I assist you today?



You:  exit


Goodbye!


In [10]:
from data import data_pipeline, preprocessing

In [14]:
data = preprocessing.get_nlp_ready_data('s_scholar_container')
    
if data:
    print(f"📄 Found {len(data)} records from Semantic Scholar")
    
    # Preprocess for Qwen
    dataset = preprocessing.preprocess_for_qwen(
        data, 
        model_name="Qwen/Qwen2.5-3B",
        max_length=1024,  # Adjust based on your needs
        save_path="data/qwen_processed_data.pkl"
    )
    
    # Show statistics
    preprocessing.get_dataset_stats(dataset)
    
    print(f"\n✅ Preprocessing complete! Dataset ready for training.")
else:
    print("❌ No data found in s_scholar_container!")

Getting credentials...
Creating Key Vault client...
Retrieving secrets...
📊 Retrieving all data from s_scholar_container...
Read 135852 items
📄 Processing 135852 records for NLP...
✅ Formatted 135852 records for NLP processing
📄 Found 135852 records from Semantic Scholar
🔄 Preprocessing 135852 records for Qwen/Qwen2.5-3B


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ Cleaned data: 135491 records
🔄 Tokenizing data...


Map:   0%|          | 0/135491 [00:00<?, ? examples/s]

✅ Tokenized 135491 samples
💾 Saving processed data to qwen_processed_data.pkl

📊 Dataset Statistics:
Total samples: 135491
Average tokens per sample: 257.1
Max tokens: 1024
Min tokens: 6

Source distribution:
  s_scholar_container: 135491 samples

✅ Preprocessing complete! Dataset ready for training.


In [1]:
from phase1_model_instantiation import open_ai

In [2]:
gpt = open_ai.GPT()

In [3]:
response = gpt.ask("Hello, how are you?")
print(response)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with anything you need. How can I assist you today?


In [4]:
print(gpt.ask("What is a transformer in machine learning?"))

A transformer is a type of deep learning model that has been very successful in natural language processing tasks, such as language translation and text generation. It is based on a self-attention mechanism that allows the model to consider the relationships between all the words in a sentence or sequence simultaneously, rather than sequentially like traditional recurrent neural networks. Transformers have significantly improved the state-of-the-art in many NLP tasks and have been widely adopted in the field.


In [2]:
!pip install azure-keyvault-secrets azure-identity azure-cosmos

  Obtaining dependency information for azure-keyvault-secrets from https://files.pythonhosted.org/packages/26/94/7c902e966b28e7cb5080a8e0dd6bffc22ba44bc907f09c4c633d2b7c4f6a/azure_keyvault_secrets-4.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for azure-cosmos from https://files.pythonhosted.org/packages/61/dc/380f843744535497acd0b85aacb59565c84fc28bf938c8d6e897a858cd95/azure_cosmos-4.9.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/80.8 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/80.8 kB ? eta -:--:--
     ---------------------------------------- 80.8/80.8 kB 1.1 MB/s eta 0:00:00
  Obtaining dependency information for azure-core>=1.31.0 from https://files.pythonhosted.org/packages/27/52/805980aa1ba18282077c484dba634ef0ede1e84eec8be9c92b2e162d0ed6/azure_core-1.35.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/46.5 kB ? eta -:--:--
     ---------------------------------------- 46.5

In [3]:
from phase2_dapt_implementation import create_qa 

In [5]:
create_qa.get_papers(3)

Getting credentials...
Creating Key Vault client...
Retrieving secrets...


DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Please run 'az login' to set up an account
	AzurePowerShellCredential: Az.Account module >= 2.2.0 is not installed
	AzureDeveloperCliCredential: Azure Developer CLI could not be found. Please visit https://aka.ms/azure-dev for installation instructions and then,once installed, authenticate to your Azure account using 'azd auth login'.
To mitigate this issue, please refer to the troubleshooting guidelines 

Error retrieving secrets: DefaultAzureCredential failed to retrieve a token from the included credentials.
Attempted credentials:
	EnvironmentCredential: EnvironmentCredential authentication unavailable. Environment variables are not fully configured.
Visit https://aka.ms/azsdk/python/identity/environmentcredential/troubleshoot to troubleshoot this issue.
	ManagedIdentityCredential: ManagedIdentityCredential authentication unavailable, no response from the IMDS endpoint.
	SharedTokenCacheCredential: SharedTokenCacheCredential authentication unavailable. No accounts were found in the cache.
	AzureCliCredential: Please run 'az login' to set up an account
	AzurePowerShellCredential: Az.Account module >= 2.2.0 is not installed
	AzureDeveloperCliCredential: Azure Developer CLI could not be found. Please visit https://aka.ms/azure-dev for installation instructions and then,once installed, authenticate to your Azure account using 'azd auth login'.
To mitigate this issue, please refer to the t

ValueError: Cosmos DB credentials not found in Key Vault